In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack, csr_matrix

In [3]:
df = pd.read_csv("csv/new_data_test.csv")

TEXT_COL = "text_clean"
LABEL_COL = "high_engagement"

In [4]:

df_model = df[[TEXT_COL, LABEL_COL]].dropna().copy()


X_train_df, X_test_df = train_test_split(
    df_model,
    test_size=0.2,
    random_state=42,
    stratify=df_model[LABEL_COL]
)

y_train = X_train_df[LABEL_COL].values
y_test  = X_test_df[LABEL_COL].values


tfidf = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 2),
    min_df=2
)

X_train = tfidf.fit_transform(X_train_df[TEXT_COL])
X_test  = tfidf.transform(X_test_df[TEXT_COL])


log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="liblinear",
    random_state=42
)

log_reg.fit(X_train, y_train)


y_pred = log_reg.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      1073
           1       0.67      0.75      0.71       368

    accuracy                           0.84      1441
   macro avg       0.79      0.81      0.80      1441
weighted avg       0.85      0.84      0.84      1441

Confusion Matrix:
[[936 137]
 [ 93 275]]


In [8]:


df = pd.read_csv("new_data_test.csv")  # your CSV file

TEXT_COL = "text_clean"
LABEL_COL = "high_engagement"  # already in your CSV


df_model = df[[TEXT_COL, LABEL_COL]].dropna().copy()
df_model["post_length"] = df_model[TEXT_COL].str.split().str.len()


X_train_df, X_test_df = train_test_split(
    df_model,
    test_size=0.2,
    random_state=42,
    stratify=df_model[LABEL_COL]
)

y_train = X_train_df[LABEL_COL].values
y_test  = X_test_df[LABEL_COL].values


tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 3),  # unigrams, bigrams, trigrams
    min_df=3,
    max_df=0.9,
    sublinear_tf=True
)

X_train_text = tfidf.fit_transform(X_train_df[TEXT_COL])
X_test_text  = tfidf.transform(X_test_df[TEXT_COL])

X_train_num = csr_matrix(X_train_df[["post_length"]].values)
X_test_num  = csr_matrix(X_test_df[["post_length"]].values)

scaler = StandardScaler(with_mean=False)
X_train_num = scaler.fit_transform(X_train_num)
X_test_num  = scaler.transform(X_test_num)

X_train = hstack([X_train_text, X_train_num])
X_test  = hstack([X_test_text, X_test_num])

param_grid = {"C": [0.01, 0.1, 0.5, 1, 5, 10]}

grid = GridSearchCV(
    LogisticRegression(
        max_iter=2000,
        solver="liblinear",
        class_weight="balanced",
        random_state=42
    ),
    param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1073
           1       0.81      0.78      0.80       368

    accuracy                           0.90      1441
   macro avg       0.87      0.86      0.86      1441
weighted avg       0.90      0.90      0.90      1441

Confusion Matrix:
[[1005   68]
 [  80  288]]
